In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mouth-map-comp/data/s1/bwam9s.mpg
/kaggle/input/mouth-map-comp/data/s1/lgbm4p.mpg
/kaggle/input/mouth-map-comp/data/s1/bbizzn.mpg
/kaggle/input/mouth-map-comp/data/s1/srwo6n.mpg
/kaggle/input/mouth-map-comp/data/s1/bgbu3s.mpg
/kaggle/input/mouth-map-comp/data/s1/pwax6p.mpg
/kaggle/input/mouth-map-comp/data/s1/srwb8n.mpg
/kaggle/input/mouth-map-comp/data/s1/sgic1a.mpg
/kaggle/input/mouth-map-comp/data/s1/sgbp6p.mpg
/kaggle/input/mouth-map-comp/data/s1/sbwo2p.mpg
/kaggle/input/mouth-map-comp/data/s1/srit9s.mpg
/kaggle/input/mouth-map-comp/data/s1/sgio6n.mpg
/kaggle/input/mouth-map-comp/data/s1/srah6p.mpg
/kaggle/input/mouth-map-comp/data/s1/prip2p.mpg
/kaggle/input/mouth-map-comp/data/s1/sbinzp.mpg
/kaggle/input/mouth-map-comp/data/s1/bgaa6n.mpg
/kaggle/input/mouth-map-comp/data/s1/pgak2n.mpg
/kaggle/input/mouth-map-comp/data/s1/pgwe7s.mpg
/kaggle/input/mouth-map-comp/data/s1/bgit7a.mpg
/kaggle/input/mouth-map-comp/data/s1/pbwp6n.mpg
/kaggle/input/mouth-map-comp/data/s1/bgw

In [13]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv3D, LSTM, Dense, Dropout, Bidirectional, MaxPool3D, BatchNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from glob import glob
import cv2

class LipReadingDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data_path, alignment_path, batch_size=16, frame_length=75, 
                 image_height=46, image_width=140, video_paths=None, alignment_paths=None, 
                 fixed_vocabulary=None, **kwargs):
        super().__init__(**kwargs)
        self.data_path = data_path
        self.alignment_path = alignment_path
        self.batch_size = batch_size
        self.frame_length = frame_length
        self.image_height = image_height
        self.image_width = image_width

        # Use provided paths or generate from directory
        if video_paths is None:
            self.video_paths = sorted(glob(os.path.join(data_path, '*.mpg')))
        else:
            self.video_paths = video_paths
        
        if alignment_paths is None:
            self.alignment_paths = sorted(glob(os.path.join(alignment_path, '*.align')))
        else:
            self.alignment_paths = alignment_paths
        
        print(f"Found {len(self.video_paths)} video files and {len(self.alignment_paths)} alignment files")
        
        # Use fixed vocabulary if provided, otherwise create from data
        if fixed_vocabulary is not None:
            self.vocabulary = fixed_vocabulary
        else:
            self.vocabulary = self._create_word_vocabulary()
            
        self.char_to_num = tf.keras.layers.StringLookup(
            vocabulary=self.vocabulary, oov_token="")
        self.num_to_char = tf.keras.layers.StringLookup(
            vocabulary=self.vocabulary, oov_token="", invert=True)

    def _create_word_vocabulary(self):
        words = set()
        print(f"Processing alignment files from: {self.alignment_path}")
        
        for align_path in self.alignment_paths:
            try:
                with open(align_path, 'r') as f:
                    content = f.read().strip().split()
                    words.update([content[i] for i in range(2, len(content), 3)])
            except Exception as e:
                print(f"Error processing {align_path}: {str(e)}")
        
        words.discard('sil')
        vocabulary = sorted(list(words))
        
        if not vocabulary:
            print("No words found in alignment files. Using default vocabulary.")
            vocabulary = ['bin', 'blue', 'at', 'f', 'two', 'now']
        
        print(f"Vocabulary size: {len(vocabulary)}")
        return vocabulary

    def __len__(self):
        return max(1, len(self.video_paths) // self.batch_size)
    
    def _process_video(self, video_path):
        frames = []
        cap = cv2.VideoCapture(video_path)
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            mouth = gray[190:236, 80:220]
            mouth = cv2.resize(mouth, (self.image_width, self.image_height))
            frames.append(mouth)
        
        cap.release()
        
        frames = np.array(frames, dtype=np.float32)
        frames = (frames - frames.mean()) / (frames.std() + 1e-6)
        
        if len(frames) < self.frame_length:
            pad_length = self.frame_length - len(frames)
            frames = np.pad(frames, ((0, pad_length), (0, 0), (0, 0)), mode='constant')
        else:
            frames = frames[:self.frame_length]
        
        return frames
    
    def _process_alignment(self, alignment_path):
        with open(alignment_path, 'r') as f:
            content = f.read().strip().split()
        
        words = [content[i] for i in range(2, len(content), 3) if content[i] != 'sil']
        text = ' '.join(words)
        return self.char_to_num(tf.convert_to_tensor(text.split()))
    
    def __getitem__(self, idx):
        batch_videos = self.video_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_alignments = self.alignment_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        X = np.zeros((len(batch_videos), self.frame_length, self.image_height, self.image_width, 1))
        Y = np.zeros((len(batch_videos), len(self.vocabulary)))
        
        for i, (video_path, align_path) in enumerate(zip(batch_videos, batch_alignments)):
            frames = self._process_video(video_path)
            X[i] = frames.reshape(self.frame_length, self.image_height, self.image_width, 1)
            
            labels = self._process_alignment(align_path)
            Y[i] = tf.reduce_max(tf.one_hot(labels, len(self.vocabulary)), axis=0)
        
        return X, Y

def build_model(frame_length, image_height, image_width, vocabulary_size):
    model = Sequential([
        tf.keras.Input(shape=(frame_length, image_height, image_width, 1)), 
        Conv3D(64, kernel_size=(3, 3, 3), activation='relu'),
        MaxPool3D(pool_size=(1, 2, 2)),
        BatchNormalization(),
        
        Conv3D(128, kernel_size=(3, 3, 3), activation='relu'),
        MaxPool3D(pool_size=(1, 2, 2)),
        BatchNormalization(),
        
        Conv3D(256, kernel_size=(3, 3, 3), activation='relu'),
        MaxPool3D(pool_size=(1, 2, 2)),
        BatchNormalization(),
        
        Reshape((-1, 256)),
        
        Bidirectional(LSTM(128, return_sequences=True)),
        Dropout(0.5),
        
        Bidirectional(LSTM(64)),
        Dropout(0.5),
        
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(vocabulary_size, activation='softmax')
    ])
    
    return model

def train_and_save_main_model(data_dir, alignment_dir, batch_size=16):
    print("Starting training for sentence-level prediction...")
    
    model_dir = "models_main"
    os.makedirs(model_dir, exist_ok=True)
    
    # Create full data generator to get consistent vocabulary
    full_data_generator = LipReadingDataGenerator(data_dir, alignment_dir, batch_size=batch_size)
    full_vocabulary = full_data_generator.vocabulary
    
    # Split video paths into two halves
    total_videos = len(full_data_generator.video_paths)
    mid_point = total_videos // 2
    
    # First half training
    first_half_generator = LipReadingDataGenerator(
        data_path=data_dir, 
        alignment_path=alignment_dir, 
        batch_size=batch_size,
        video_paths=full_data_generator.video_paths[:mid_point],
        alignment_paths=full_data_generator.alignment_paths[:mid_point],
        fixed_vocabulary=full_vocabulary
    )
    
    # Build and compile the model
    model = build_model(
        frame_length=75,
        image_height=46,
        image_width=140,
        vocabulary_size=len(full_vocabulary)
    )
    
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Callbacks for first half
    first_half_callbacks = [
        ModelCheckpoint(
            os.path.join(model_dir, 'lip_reading_first_half_best.keras'),
            save_best_only=True,
            monitor='accuracy'
        ),
        EarlyStopping(
            monitor='loss',
            patience=5,
            restore_best_weights=True
        )
    ]

    # Train on first half
    print("Training on first half of the dataset...")
    model.fit(
        first_half_generator,
        epochs=10,
        callbacks=first_half_callbacks
    )
    
    # Second half training
    second_half_generator = LipReadingDataGenerator(
        data_path=data_dir, 
        alignment_path=alignment_dir, 
        batch_size=batch_size,
        video_paths=full_data_generator.video_paths[mid_point:],
        alignment_paths=full_data_generator.alignment_paths[mid_point:],
        fixed_vocabulary=full_vocabulary
    )
    
    # Callbacks for second half
    second_half_callbacks = [
        ModelCheckpoint(
            os.path.join(model_dir, 'lip_reading_full_dataset_best.keras'),
            save_best_only=True,
            monitor='accuracy'
        ),
        EarlyStopping(
            monitor='loss',
            patience=5,
            restore_best_weights=True
        )
    ]

    # Train on second half
    print("Training on second half of the dataset...")
    model.fit(
        second_half_generator,
        epochs=10,
        callbacks=second_half_callbacks
    )

    # Save final model
    final_model_path = os.path.join(model_dir, 'lip_reading_full_model.h5')
    model.save(final_model_path)
    print(f"Final model saved: {final_model_path}")

    # Save vocabulary
    vocab_path = os.path.join(model_dir, 'vocabulary_main.txt')
    with open(vocab_path, 'w') as f:
        f.write('\n'.join(full_vocabulary))
    print(f"Vocabulary saved: {vocab_path}")

if __name__ == "__main__":
    data_dir = r"/kaggle/input/mouth-map-comp/data/s1"
    alignment_dir = r"/kaggle/input/mouth-map-comp/data/alignments/s1"
    
    print("Training main model...")
    train_and_save_main_model(data_dir, alignment_dir)

Training main model...
Starting training for sentence-level prediction...
Found 1000 video files and 1000 alignment files
Processing alignment files from: /kaggle/input/mouth-map-comp/data/alignments/s1
Vocabulary size: 52
Found 500 video files and 500 alignment files
Training on first half of the dataset...
Epoch 1/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 0.0294 - loss: 23.3035
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.1069 - loss: 22.7980
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.1268 - loss: 22.9664
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.0864 - loss: 24.5097
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.1376 - loss: 27.4138
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.0840 - loss: 32.5829
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.0678 - loss: 35.8482
Found 500 video files and 500 alignment files
Training on second half of the dataset...
Epoch 1